# Projet Steam
## Flattening a nested schema

In [0]:
items_exploded_path = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"

initial_df = spark.read.json(items_exploded_path)

Count the rows in the DataFrame

In [0]:
initial_df.count()

Out[2]: 55691

In [0]:
initial_df.columns

Out[3]: ['data', 'id']

The schema of the DataFrame

In [0]:
initial_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

### Working with the schema

In [0]:
# we'll work the schema in json format
initial_df.schema.jsonValue()

Out[5]: {'type': 'struct',
 'fields': [{'name': 'data',
   'type': {'type': 'struct',
    'fields': [{'name': 'appid',
      'type': 'long',
      'nullable': True,
      'metadata': {}},
     {'name': 'categories',
      'type': {'type': 'array', 'elementType': 'string', 'containsNull': True},
      'nullable': True,
      'metadata': {}},
     {'name': 'ccu', 'type': 'long', 'nullable': True, 'metadata': {}},
     {'name': 'developer', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': 'discount', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': 'genre', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': 'header_image',
      'type': 'string',
      'nullable': True,
      'metadata': {}},
     {'name': 'initialprice',
      'type': 'string',
      'nullable': True,
      'metadata': {}},
     {'name': 'languages', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': 'name', 'type': 'string', 'nullable': True, 'metadata'

# Selection et mise en forme des données dans un DataFrame 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Selection des colonnes retenues
# Mises en forme en fonction des cas :
# - string => Double
# - string en date
# - Split de valeurs multiples séparées par des "," par des listes
final_df = initial_df \
          .withColumn("appid", F.col("data.appid")) \
          .withColumn("categories", F.col("data.categories")) \
          .withColumn("ccu", F.col("data.ccu")) \
          .withColumn("developer", F.col("data.developer")) \
          .withColumn("discount", F.col("data.discount").cast(DoubleType())) \
          .withColumn("genres", F.split("data.genre", ", ")) \
          .withColumn("header_image", F.col("data.header_image")) \
          .withColumn("initialprice", F.col("data.initialprice").cast(DoubleType())) \
          .withColumn("languages", F.split("data.languages", ", ")) \
          .withColumn("name", F.col("data.name")) \
          .withColumn("negative", F.col("data.negative")) \
          .withColumn("owners", F.col("data.owners")) \
          .withColumn("platform_linux", F.col("data.platforms.linux")) \
          .withColumn("platform_mac", F.col("data.platforms.mac")) \
          .withColumn("platform_windows", F.col("data.platforms.windows")) \
          .withColumn("positive", F.col("data.positive")) \
          .withColumn("price", F.col("data.price").cast(DoubleType())) \
          .withColumn("publisher", F.col("data.publisher")) \
          .withColumn("release_date", F.to_date(F.col("data.release_date"), 'yyyy/MM/d')) \
          .withColumn("required_age", F.col("data.required_age").cast(IntegerType())) \
          .withColumn("type", F.col("data.type")) \
          .withColumn("website", F.col("data.website")) \
          .drop("data")

# Transformer les prix en montant en $ au lieu de cts et le "discount" en ratio au lieu de xx%
final_df = final_df \
          .withColumn("discount", final_df.discount / 100) \
          .withColumn("initialprice", final_df.initialprice / 100) \
          .withColumn("price", final_df.price / 100)

In [0]:
display(final_df.describe())

summary,id,appid,ccu,developer,discount,header_image,initialprice,name,negative,owners,positive,price,publisher,required_age,type,website
count,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55691,55688,55691,55691
mean,1025603.0926720655,1025603.0926720655,138.9596164550825,67392.0,0.026037779892621713,null,7.975663033519214,Infinity,241.8376937027527,null,1470.8755992889335,7.732849832104521,2001.0,0.1978882344490734,null,null
stddev,522784.96832834504,522784.96832834504,6002.067909130765,210681.70504552333,0.12887080174743096,null,11.047624778413402,NaN,5765.413761559615,null,30982.733479534887,10.931345827234507,1921.8937275510318,2.2962924614818236,null,null
min,10,10,0,,0.0,https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513,0.0,Fieldrunners 2,0,"0 .. 20,000",0,0.0,,0,game,
max,999990,2190950,874053,＼上／,0.9,https://cdn.akamai.steamstatic.com/steam/apps/999990/header.jpg?t=1610733322,999.0,～Daydream～蝶が舞う頃に,908515,"500,000 .. 1,000,000",5943345,999.0,Ｌｅｍｏｎ Ｂａｌｍ,180,hardware,www.windybeard.com


Databricks visualization. Run in Databricks to view.

###	Quel éditeur a publié le plus de jeux sur Steam ?

In [0]:
display(final_df \
        .groupBy("publisher").agg(F.count("*").alias("games_count")) \
        .orderBy(F.desc("games_count")) \
        .limit(8)
        )

publisher,games_count
Big Fish Games,422
8floor,202
SEGA,165
Strategy First,151
Square Enix,141
Choice of Games,140
Sekai Project,132
HH-Games,132


Databricks visualization. Run in Databricks to view.

### Quels sont les jeux les mieux notés ?

In [0]:
display(final_df \
        .select("appid", "name", "positive") \
        .orderBy(F.desc("positive")) \
        .limit(10)
        )

appid,name,positive
730,Counter-Strike: Global Offensive,5943345
570,Dota 2,1534895
271590,Grand Theft Auto V,1229265
578080,PUBG: BATTLEGROUNDS,1185361
105600,Terraria,1014711
359550,Tom Clancy's Rainbow Six Siege,942910
4000,Garry's Mod,861240
440,Team Fortress 2,846407
252490,Rust,732513
550,Left 4 Dead 2,643836


Databricks visualization. Run in Databricks to view.

### Y a-t-il des années avec plus de sorties ?
### Y a-t-il eu plus ou moins de sorties de jeux pendant le Covid, par exemple ?

In [0]:
display(final_df \
        .withColumn("release_year", F.year("release_date")) \
        .groupBy("release_year").agg(F.count("*").alias("games_count")) \
        .orderBy(F.desc("release_year")) \
        .limit(10)
        )

release_year,games_count
2022,7451
2021,8805
2020,8287
2019,6949
2018,7663
2017,6006
2016,4176
2015,2566
2014,1550
2013,469


Databricks visualization. Run in Databricks to view.

###	Comment les prix sont-ils distribués ?
### Y a-t-il beaucoup de jeux à prix réduit ?

In [0]:
final_df.filter(final_df.price >= 0).count()
# filter les jeux à plus de 100 (au nombre de 37 / 55000)
# pour afficher la répartition du nombre de jeux par tranche de prix sur un graphique lisible

Out[22]: 55691

In [0]:

# display(final_df \
#        .select("price") \
#        .filter(final_df.price < 100) \
#        .orderBy(F.desc("price"))
#        )    

display(final_df \
        .withColumn("cat_price",
        F.when(F.col("price") == 0, "a - 0$")
        .when((F.col("price") > 0) & (F.col("price") < 10), "b - moins 10$")
        .when((F.col("price") >= 10) & (F.col("price") < 20), "c - 10 à 20$")
        .when((F.col("price") >= 20) & (F.col("price") < 50), "d - 20 à 50$")
        .when((F.col("price") >= 50) & (F.col("price") < 100), "e - 50 à 100$")
        .when((F.col("price") >= 100), "f - 100$ et plus")
        .otherwise("x - nc")) \
        .groupBy("cat_price").agg(F.count("*").alias("games_count")) \
        .orderBy("cat_price") \
        )


# Visualisation sur un graphique de type "Histogramme"

cat_price,games_count
a - 0$,7780
b - moins 10$,35928
c - 10 à 20$,9022
d - 20 à 50$,2633
e - 50 à 100$,291
f - 100$ et plus,37


Databricks visualization. Run in Databricks to view.

### Quelles sont les langues les plus représentées ?

In [0]:
# il y a plusieurs langues par jeux
# nécessité de création d'un nouveau dataframe avec un explode de la donnée langue

display(final_df \
        .withColumn("language", F.explode("languages")) \
        .groupBy("language").agg(F.count("*").alias("games_count")) \
        .orderBy(F.desc("games_count")) \
        .limit(10)
        )

language,games_count
English,55116
German,14019
French,13426
Russian,12922
Simplified Chinese,12782
Spanish - Spain,12233
Japanese,10368
Italian,9304
Portuguese - Brazil,6750
Korean,6599


Databricks visualization. Run in Databricks to view.

### Y a-t-il beaucoup de jeux interdits aux enfants de moins de 16/18 ans ?

In [0]:
display(final_df \
        .withColumn("mois16", final_df.required_age >= 16) \
        .groupBy("mois16").agg(F.count("*").alias("games_count")) \
        .orderBy("mois16") \
        )

# A finaliser - Cadrer la question 16/18 ans

mois16,games_count
null,3
false,55383
true,305


Databricks visualization. Run in Databricks to view.

### Quels sont les genres les plus représentés ?

In [0]:
# Un jeu peux faire référence à plusieurs genre
# nécessité de création d'un nouveau dataframe avec un explode de la donnée genre

display(final_df \
        .withColumn("genre", F.explode("genres")) \
        .groupBy("genre").agg(F.count("*").alias("games_count")) \
        .orderBy(F.desc("games_count")) \
        .limit(10)
        )

genre,games_count
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895
Simulation,10836
RPG,9534
Early Access,6145
Free to Play,3393
Sports,2666


Databricks visualization. Run in Databricks to view.

### •	Y a-t-il des genres qui ont un meilleur ratio d'examen positif/négatif ?

In [0]:
# Un jeu peux faire référence à plusieurs genre
# nécessité de création d'un nouveau dataframe avec un explode de la donnée genre

display(final_df \
        .withColumn("genre", F.explode("genres")) \
        .groupBy("genre").agg((F.sum("positive")/F.sum("negative")).alias("ratio_positive_negative")) \
        .orderBy(F.desc("ratio_positive_negative")) \
        .limit(7)
)

genre,ratio_positive_negative
Photo Editing,42.03353946889778
Animation & Modeling,26.17327220369809
Design & Illustration,24.958603325063873
Utilities,16.99503482518447
Game Development,8.38759926695174
Indie,7.67017877344188
Audio Production,7.331141281289775


Databricks visualization. Run in Databricks to view.

### Certains éditeurs ont-ils des genres préférés ?

In [0]:
# Un jeu peux faire référence à plusieurs genre
# nécessité de création d'un nouveau dataframe avec un explode de la donnée genre

df1 = final_df \
     .withColumn("genre", F.explode("genres")) \
     .groupBy("publisher", "genre").agg(F.count("*").alias("genre_total")) \
     .withColumnRenamed("publisher", "publisher_exp")

df2 = final_df \
     .groupBy("publisher").agg(F.count("*").alias("total"))

# Limiter les éditeurs (50 jeux minimun)
# éviter des editeurs avec des genres préférés avec peu de jeux
df2 = df2.filter(df2.total > 15)

df_join = df1.join(df2, df1['publisher_exp'] == df2['publisher'], "inner") \
          .withColumn("ratio", F.col("genre_total")/F.col("total")) \
          .orderBy(F.desc("ratio"), F.desc("genre_total"))

display(df_join.limit(20).select(["publisher", "genre", "genre_total", "total", "ratio"]).toPandas())


publisher,genre,genre_total,total,ratio
8floor,Casual,202,202,1.0
HH-Games,Casual,132,132,1.0
Hosted Games,RPG,79,79,1.0
Hosted Games,Indie,79,79,1.0
Boogygames Studios,Indie,78,78,1.0
Tero Lunkka,Indie,68,68,1.0
RewindApp,Indie,59,59,1.0
Ripknot Systems,Casual,55,55,1.0
Hede,Action,54,54,1.0
Hede,Indie,54,54,1.0


Databricks visualization. Run in Databricks to view.

### Quels sont les genres les plus lucratifs ?

In [0]:
# A cadrer - le cas échéant - pas sur qu'il y a les infos pour répondre à la question

### •	La plupart des jeux sont-ils disponibles sur Windows/Mac/Linux ?

In [0]:
display(final_df \
        .groupBy("platform_windows", "platform_mac", "platform_linux").agg(F.count("*").alias("total")) \
        .orderBy(F.desc("platform_windows"), F.desc("platform_mac"), F.desc("platform_linux"))
       )      

platform_windows,platform_mac,platform_linux,total
true,true,true,6807
true,true,false,5951
true,false,true,1647
true,false,false,41271
false,true,true,1
false,true,false,11
false,false,true,3


Databricks visualization. Run in Databricks to view.